In [90]:
!nvidia-smi

Sat Sep  2 16:00:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          On   | 00000000:06:00.0 Off |                    0 |
|  0%   67C    P0    75W / 150W |   5906MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [91]:
!pip install transformers -U -qq
!pip install sentencepiece -U -qq

In [92]:
# Import the pandas library for data manipulation and analysis
import pandas as pd

# Import the numpy library for numerical computing
import numpy as np

# Import the Matplotlib library for creating visualizations such as plots, graphs, etc.
import matplotlib.pyplot as plt

# Import the pathlib library for working with file paths in a way that is cross-platform
from pathlib import Path

# Import functions for metrics computation like confusion matrix, and accuracy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report

# Import the transformers library for state-of-the-art Natural Language Processing (NLP) models like BERT, GPT, etc.
from transformers import pipeline

In [93]:
# Import the torch library. PyTorch is a Python library for deep learning.
import torch

# Check if a CUDA-enabled GPU is available for PyTorch.
# This can speed up neural network computations.
torch.cuda.is_available()

True

In [94]:
!pip show transformers

Name: transformers
Version: 4.32.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/ubuntu/.local/lib/python3.8/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [95]:

import sys
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install -U nltk -qq
    !pip install -U spacy -qq
    !python -m spacy download en_core_web_sm -qq

    basepath = '/content/drive/MyDrive/Projects/drug_review'
    sys.path.append('/content/drive/MyDrive/Projects/drug_review')

else:
    basepath = './'



# location of train and test files

base_folder = Path(basepath)

#train_path  = base_folder / 'drugtrain_filter.csv'
test_path = base_folder / 'drugtest_filter.csv'

#train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)




In [96]:
help(pipeline)

Help on function pipeline in module transformers.pipelines:

pipeline(task: str = None, model: Union[str, ForwardRef('PreTrainedModel'), ForwardRef('TFPreTrainedModel'), NoneType] = None, config: Union[str, transformers.configuration_utils.PretrainedConfig, NoneType] = None, tokenizer: Union[str, transformers.tokenization_utils.PreTrainedTokenizer, ForwardRef('PreTrainedTokenizerFast'), NoneType] = None, feature_extractor: Union[str, ForwardRef('SequenceFeatureExtractor'), NoneType] = None, image_processor: Union[str, transformers.image_processing_utils.BaseImageProcessor, NoneType] = None, framework: Union[str, NoneType] = None, revision: Union[str, NoneType] = None, use_fast: bool = True, token: Union[str, bool, NoneType] = None, device: Union[int, str, ForwardRef('torch.device'), NoneType] = None, device_map=None, torch_dtype=None, trust_remote_code: Union[bool, NoneType] = None, model_kwargs: Dict[str, Any] = None, pipeline_class: Union[Any, NoneType] = None, **kwargs) -> transform

In [97]:
# Load zero shot classification pipleine - do not pass any model. We will use the default model
# use pytorch as framework and move the pipleine to GPU
# syntax : zero_shot_classifier = pipeline(name-of-pipeline, framework=, device =)

zero_shot_classifier = pipeline("zero-shot-classification",
                      framework='pt',
                     device = 0)

In [98]:
test_df_sample = test_df.sample(frac = 0.01, random_state = 21 )
test_df_sample

,uniqueID,drugName,condition,review,rating,date,usefulCount
33381,153869,Amoxicillin,Bacterial Infection,"""I was awake with a rampant earache and crying...",10,02-Oct-08,30
38953,161733,Remeron,Depression,"""Remeron helped very much initially. After try...",2,03-Nov-12,59
49004,207475,Suprep Bowel Prep Kit,Bowel Preparation,"""Not bad at all!! I&#039;ve had drinks at the ...",10,01-Apr-17,14
27277,3355,Nitrofurantoin,Urinary Tract Infection,"""I took this medicine to treat a UTI. Althoug...",4,31-Dec-15,8
9950,199172,Depo-Provera,Birth Control,"""Ever since I took this shot I have been havin...",1,04-Dec-15,5
...,...,...,...,...,...,...,...
44780,174594,Abreva,Herpes Simplex,"""I get cold sores about 2-3 times per year. Ab...",8,08-Dec-15,8
19897,44693,Catapres,Opiate Withdrawal,"""Clonidine is wonderful for me. I&#039;ve been...",7,28-Apr-15,132
8873,158715,Methylphenidate,ADHD,"""Better than Adderall for me. Adderall feels l...",9,27-Nov-10,20
35731,150181,Mucinex D,Cough and Nasal Congestion,"""I got the generic, of Mucenix DM. It has defi...",2,22-Sep-14,46


In [99]:
# create a list of reviews (hint use column.tolist() to convert numpy array series to python list)
texts_train = test_df_sample['review'].tolist()  # CODE HERE

# list of labels
candidate_labels = test_df_sample['condition'].unique().tolist() # CODE HERE
candidate_labels

['Bacterial Infection',
 'Depression',
 'Bowel Preparation',
 'Urinary Tract Infection',
 'Birth Control',
 'Rosacea',
 'Urticaria',
 'Irritable Bowel Syndrome',
 'Asthma, Maintenance',
 'Panic Disorde',
 'Erectile Dysfunction',
 'Headache',
 'Pain',
 'Bladder Infection',
 'Onychomycosis, Toenail',
 'Anxiety',
 'Obesity',
 'Osteoarthritis',
 'Allergies',
 'Seizure Prevention',
 'Vaginal Yeast Infection',
 'Chronic Fatigue Syndrome',
 'Hot Flashes',
 'Cold Sores',
 'ADHD',
 'Nausea/Vomiting',
 'Insomnia',
 'Emergency Contraception',
 'Postmenopausal Symptoms',
 'Major Depressive Disorde',
 'Bipolar Disorde',
 'Psoriasis',
 'ibromyalgia',
 'Migraine Prevention',
 'High Blood Pressure',
 'Obsessive Compulsive Disorde',
 'Osteoporosis',
 'Chlamydia Infection',
 'Muscle Spasm',
 'Acne',
 'Diabetes, Type 2',
 'Cough and Nasal Congestion',
 'Skin or Soft Tissue Infection',
 'Anxiety and Stress',
 'Epilepsy',
 'Hepatitis C',
 'Keratosis',
 'Hypogonadism, Male',
 'Helicobacter Pylori Infection'

In [100]:
help(zero_shot_classifier)

Help on ZeroShotClassificationPipeline in module transformers.pipelines.zero_shot_classification object:

class ZeroShotClassificationPipeline(transformers.pipelines.base.ChunkPipeline)
 |  ZeroShotClassificationPipeline(args_parser=<transformers.pipelines.zero_shot_classification.ZeroShotClassificationArgumentHandler object at 0x7f4f969e6be0>, *args, **kwargs)
 |  
 |  NLI-based zero-shot classification pipeline using a `ModelForSequenceClassification` trained on NLI (natural
 |  language inference) tasks. Equivalent of `text-classification` pipelines, but these models don't require a
 |  hardcoded number of potential classes, they can be chosen at runtime. It usually means it's slower but it is
 |  **much** more flexible.
 |  
 |  Any combination of sequences and labels can be passed and each combination will be posed as a premise/hypothesis
 |  pair and passed to the pretrained model. Then, the logit for *entailment* is taken as the logit for the candidate
 |  label being valid. Any

In [101]:
from transformers.utils import logging

logging.set_verbosity(50)

In [102]:
logging.get_verbosity()

50

In [103]:
logging.set_verbosity_error()

In [78]:
probs = zero_shot_classifier(sequences = texts_train, candidate_labels = candidate_labels)

/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f4f3baf0280> was reported to be 514 (when accessing len(dataloader)), but 515 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f4f3baf0280> was reported to be 514 (when accessing len(dataloader)), but 516 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f4f3baf0280> was reported to be 514 (when accessing len(dataloader)), but 517 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py:640: UserWarning: Le

In [79]:
probs[0]

In [80]:
predictions =  [review['labels'][np.argmax(review['scores'])] for review in probs]

In [81]:
from sklearn.metrics import *
accuracy =  accuracy_score(predictions, test_df_sample['condition'].tolist())*100
accuracy